In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

In [2]:
data_path = '/Users/sola/Downloads/open/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [3]:
train = train.fillna(0)
test = test.fillna(0)

In [4]:
# 모든 값이 0인 피처 지울거임..
wrong = []
for i in range(1,2876):
    ch = 'X_'+str(i)
    if train[ch].mean() == 0:
        wrong.append(ch)

print(wrong)
print(len(wrong))

['X_3', 'X_6', 'X_14', 'X_70', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_82', 'X_83', 'X_84', 'X_85', 'X_142', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_259', 'X_260', 'X_315', 'X_319', 'X_322', 'X_323', 'X_324', 'X_327', 'X_328', 'X_329', 'X_330', 'X_375', 'X_376', 'X_377', 'X_378', 'X_427', 'X_466', 'X_467', 'X_505', 'X_508', 'X_558', 'X_561', 'X_567', 'X_583', 'X_617', 'X_620', 'X_631', 'X_633', 'X_634', 'X_636', 'X_638', 'X_639', 'X_640', 'X_641', 'X_642', 'X_672', 'X_673', 'X_676', 'X_691', 'X_692', 'X_695', 'X_715', 'X_729', 'X_732', 'X_743', 'X_749', 'X_759', 'X_760', 'X_761', 'X_764', 'X_776', 'X_777', 'X_778', 'X_836', 'X_843', 'X_844', 'X_849', 'X_859', 'X_886', 'X_887', 'X_888', 'X_889', 'X_934', 'X_935', 'X_936', 'X_937', 'X_992', 'X_1020', 'X_1021', 'X_1022', 'X_1023', 'X_1024', 'X_1025', 'X_1070', 'X_1092', 'X_1119', 'X_1137', 'X_1146', 'X_1206', 'X_1216', 'X_1219', 'X_1248', 'X_1249', 'X_1250', 'X_1251', 'X_1252', 'X_125

In [5]:
train = train.drop(wrong, axis=1)
test = test.drop(wrong, axis=1)

train = train.drop(['TIMESTAMP','PRODUCT_ID'], axis=1)
test = test.drop(['TIMESTAMP','PRODUCT_ID'], axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# 수치형으로 변환
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])

    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i])
print('Done.')

Done.


In [10]:
train_x = train.drop(['Y_Class', 'Y_Quality'], axis=1)
y = train['Y_Class']

X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.3, random_state=37)

In [11]:
from sklearn.ensemble import RandomForestClassifier

feat_labels = X_train.columns

hgb = HistGradientBoostingClassifier(random_state=37)
scores = cross_validate(hgb, X_train, y_train, return_train_score=True, n_jobs=-1)
# print(np.mean(scores['train_score']), np.mean(scores['test_score']))

hgb.fit(X_train, y_train)
#permutation_importance는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산한다.
#n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정한다.
result = permutation_importance(hgb, X_train, y_train, n_repeats=5, random_state=37, n_jobs=-1)
# print(result.importances_mean)

importances = result.importances

# argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
indices = np.argsort(importances)[::-1]

# for i in range(X_train.shape[1]):
	# print(feat_labels[indices[i]], importances[indices[i]])

#상위20개만...
a=[]
for i in range(100):
    a.append(feat_labels[indices[i]])
print(a)

[Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_4'], dtype='object'), Index(['LINE', 'PRO

In [ ]:
train_x = train_x[a]
y = train['Y_Class']
test_x = test[a]

X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.3, random_state=37)

rf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

prediction = rf.predict(X_valid)

rf.score(X_valid, y_valid)

In [9]:
train_x = train.drop(['Y_Class', 'Y_Quality'], axis=1)
y = train['Y_Class']
X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.3, stratify=y, random_state=37)

hgb = HistGradientBoostingClassifier(random_state=37)
scores = cross_validate(hgb, X_train, y_train, return_train_score=True, n_jobs=-1)
# print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#0.9321723946453317 0.8801241948619236

hgb.fit(X_train, y_train)
#permutation_importance는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산한다.
#n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정한다.
result = permutation_importance(hgb, X_train, y_train, n_repeats=5, random_state=37, n_jobs=-1)
print(result.importances_mean)

#[0.08876275 0.23438522 0.08027708]#[0.08876275 0.23438522 0.08027708]


# result = permutation_importance(hgb, X_train, y_train, n_repeats=10, random_state=42, n_jobs=-1)
# print(result.importances_mean)
#[0.05969231 0.20238462 0.049     ]


# hgb.score(X_train, y_train)
#0.8723076923076923

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
RF = RandomForestClassifier(n_jobs = -1 ,random_state=37)

In [ ]:
RF.fit(train_x, y)
preds = RF.predict(test_x)

In [ ]:
preds

In [ ]:
submit = pd.read_csv('/Users/sola/Downloads/open/sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('/Users/sola/Downloads/open/baseline_submission10.csv', index=False)

랜덤포레스트 정확도

In [12]:
rf = RandomForestClassifier(random_state=37)
X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.3, random_state=37)

rf.fit(X_train, y_train)

y_pred=rf.predict(X_valid)

In [13]:
from sklearn.metrics import accuracy_score

print(f'랜덤포레스트 정확도 : {accuracy_score(y_valid, y_pred)*100:.2f}%')

랜덤포레스트 정확도 : 77.22%
